Imports


In [1]:
import torch
import torchvision
import pandas as pd
import torch.nn as nn
from tqdm import tqdm
import multiprocessing
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import tabulate

Versión torch


In [2]:
print("Torch version: ", torch.__version__)

####################################################################
# Set Device
####################################################################

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device: ", device)

Torch version:  2.5.1+cu121
Device:  cuda


Preparar datos


In [3]:
####################################################################
# Prepare Data
####################################################################

train_set = torchvision.datasets.MNIST(".data/", train=True, download=True)
# train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)

test_set = torchvision.datasets.MNIST(".data/", train=False, download=True)
# test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=True)

print("Train images: ", train_set)
print("Image: ", train_set[0][0])
print("Label: ", train_set[0][1])
print("Label one hot: ", F.one_hot(torch.tensor(train_set[0][1]), num_classes=10))

Using downloaded and verified file: .data/MNIST/raw/train-images-idx3-ubyte.gz
Extracting .data/MNIST/raw/train-images-idx3-ubyte.gz to .data/MNIST/raw

Using downloaded and verified file: .data/MNIST/raw/train-labels-idx1-ubyte.gz
Extracting .data/MNIST/raw/train-labels-idx1-ubyte.gz to .data/MNIST/raw

Using downloaded and verified file: .data/MNIST/raw/t10k-images-idx3-ubyte.gz
Extracting .data/MNIST/raw/t10k-images-idx3-ubyte.gz to .data/MNIST/raw

Failed to download (trying next):
<urlopen error [Errno 110] Connection timed out>



100%|██████████| 4.54k/4.54k [00:00<00:00, 3.20MB/s]

Extracting .data/MNIST/raw/t10k-labels-idx1-ubyte.gz to .data/MNIST/raw

Train images:  Dataset MNIST
    Number of datapoints: 60000
    Root location: .data/
    Split: Train
Image:  <PIL.Image.Image image mode=L size=28x28 at 0x7FDD5E9B74C0>
Label:  5
Label one hot:  tensor([0, 0, 0, 0, 0, 1, 0, 0, 0, 0])


Dataset class


In [4]:
####################################################################
# Dataset Class
####################################################################
class MNIST_dataset(Dataset):
    def __init__(self, data, partition="train"):
        print("\nLoading MNIST ", partition, " Dataset...")
        self.data = data
        self.partition = partition
        print("\tTotal Len.: ", len(self.data), "\n", 50 * "-")

    def __len__(self):
        return len(self.data)

    def from_pil_to_tensor(self, image):
        return torchvision.transforms.ToTensor()(image)

    def __getitem__(self, idx):
        # Image
        image = self.data[idx][0]
        # PIL Image to torch tensor
        image_tensor = self.from_pil_to_tensor(image)
        # care! net expect a 784 size vector and our dataset
        # provide 1x28x28 (channels, height, width) -> Reshape!
        image_tensor = image_tensor.view(-1)

        # Label
        label = torch.tensor(self.data[idx][1])
        label = F.one_hot(label, num_classes=10).float()

        return {"img": image_tensor, "label": label}


train_set = MNIST_dataset(train_set, partition="train")
test_set = MNIST_dataset(test_set, partition="test")


Loading MNIST  train  Dataset...
	Total Len.:  60000 
 --------------------------------------------------

Loading MNIST  test  Dataset...
	Total Len.:  10000 
 --------------------------------------------------


DataLoader class


In [5]:
####################################################################
# DataLoader Class
####################################################################

batch_size = 100
num_workers = multiprocessing.cpu_count() - 1
print("Num workers", num_workers)
train_dataloader = DataLoader(
    train_set, batch_size, shuffle=True, num_workers=num_workers
)
test_dataloader = DataLoader(
    test_set, batch_size, shuffle=False, num_workers=num_workers
)

Num workers 1


NN Class


In [6]:
####################################################################
# Neural Network Class
####################################################################
# Creating our Neural Network - Fully Connected
class Net(nn.Module):
    def __init__(self, num_classes):
        super(Net, self).__init__()
        self.linear1 = nn.Linear(784, 1024)
        self.relu1 = F.gelu
        self.linear2 = nn.Linear(1024, 1024)
        self.relu2 = F.gelu
        #self.linear3 = nn.Linear(1024, 1024)
        #self.relu3 = F.gelu
        self.classifier = nn.Linear(1024, num_classes)

    def forward(self, x):
        out = self.relu1(self.linear1(x))
        out = self.relu2(self.linear2(out))
        #out = self.relu3(self.linear3(out))
        out = self.classifier(out)
        return out


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

Función de entrenamineto


In [7]:
# TRAIN NETWORK
def train_network(optimizer, epoch, criterion, net):
    train_loss, train_correct = 0, 0
    net.train()
    with tqdm(
        iter(train_dataloader), desc="Epoch " + str(epoch), unit="batch"
    ) as tepoch:
        for batch in tepoch:
            # Returned values of Dataset Class
            images = batch["img"].to(device)
            labels = batch["label"].to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # Forward
            outputs = net(images)
            loss = criterion(outputs, labels)

            # Calculate gradients
            loss.backward()

            # Update parameters
            optimizer.step()

            # one hot -> labels
            labels = torch.argmax(labels, dim=1)
            pred = torch.argmax(outputs, dim=1)
            train_correct += pred.eq(labels).sum().item()

            # print statistics
            train_loss += loss.item()

    train_loss /= len(train_dataloader.dataset)
    return train_loss, train_correct

Función de test


In [8]:
# TEST NETWORK
def test_network(net, epoch, criterion):
    test_loss, test_correct = 0, 0
    net.eval()
    with torch.no_grad():
        with tqdm(
            iter(test_dataloader), desc="Test " + str(epoch), unit="batch"
        ) as tepoch:
            for batch in tepoch:
                images = batch["img"].to(device)
                labels = batch["label"].to(device)

                # Forward
                outputs = net(images)
                test_loss += criterion(outputs, labels)

                # one hot -> labels
                labels = torch.argmax(labels, dim=1)
                pred = torch.argmax(outputs, dim=1)

                test_correct += pred.eq(labels).sum().item()

        test_loss /= len(test_dataloader.dataset)
        test_accuracy = 100.0 * test_correct / len(test_dataloader.dataset)
    return test_loss, test_correct, test_accuracy

Entrenamiento con validación


In [9]:
# Instantiating the network and printing its architecture
num_classes = 10
net = Net(num_classes)
print(net)
print("Params: ", count_parameters(net))
####################################################################
# Training settings
####################################################################

# Training hyperparameters
epochs = 15
criterion = nn.CrossEntropyLoss()
best_results = dict()
for lr in [0.1, 0.01, 0.001, 0.0001]:
    best_results[lr] = dict()
    for optimizer in [optim.AdamW, optim.SGD, optim.Adadelta, optim.Adam]:
        ####################################################################
        # Training
        ####################################################################

        # Load model in GPU, cada vez que hacemos un entrenamiento con un tipo de optimizador y lr tenemos que crear la red
        net = Net(num_classes)
        net.to(device)
        # y el optimizador debe crearse despues de haber creado la red porque le pasamos los parámetros de la red que serán los que se encargue de optimizar
        if optimizer == optim.SGD:
            optimizer = optimizer(
                net.parameters(), lr=lr, weight_decay=1e-6, momentum=0.9
            )
        else:
            optimizer = optimizer(net.parameters(), lr=lr, weight_decay=1e-6)

        best_results[lr][optimizer.__class__.__name__] = {
            "best_accuracy": -1,
            "best_epoch": 0,
        }

        print("\n---- Start Training ----")
        print(f"Optimizer: {optimizer.__class__.__name__} - Learning Rate: {lr}")
        for epoch in range(epochs):
            train_loss, train_correct = train_network(optimizer, epoch, criterion, net)

            test_loss, test_correct, test_accuracy = test_network(net, epoch, criterion)

            print(
                "[Epoch {}] Train Loss: {:.6f} - Test Loss: {:.6f} - Train Accuracy: {:.2f}% - Test Accuracy: {:.2f}%".format(
                    epoch + 1,
                    train_loss,
                    test_loss,
                    100.0 * train_correct / len(train_dataloader.dataset),
                    test_accuracy,
                )
            )

            if (
                test_accuracy
                > best_results[lr][optimizer.__class__.__name__]["best_accuracy"]
            ):
                best_results[lr][optimizer.__class__.__name__][
                    "best_accuracy"
                ] = test_accuracy
                best_results[lr][optimizer.__class__.__name__]["best_epoch"] = epoch

                # Save best weights
                torch.save(net.state_dict(), "best_model.pt")

        print(
            "\nBEST TEST ACCURACY: ",
            best_results[lr][optimizer.__class__.__name__]["best_accuracy"],
            " in epoch ",
            best_results[lr][optimizer.__class__.__name__]["best_epoch"],
        )

####################################################################
# Results
####################################################################
print(best_results)

Net(
  (linear1): Linear(in_features=784, out_features=1024, bias=True)
  (linear2): Linear(in_features=1024, out_features=1024, bias=True)
  (classifier): Linear(in_features=1024, out_features=10, bias=True)
)
Params:  1863690

---- Start Training ----
Optimizer: AdamW - Learning Rate: 0.1


Test 0: 100%|██████████| 100/100 [00:02<00:00, 35.75batch/s]

[Epoch 1] Train Loss: 0.237297 - Test Loss: 0.004571 - Train Accuracy: 81.09% - Test Accuracy: 87.84%



Test 1: 100%|██████████| 100/100 [00:02<00:00, 45.27batch/s]

[Epoch 2] Train Loss: 0.005127 - Test Loss: 0.005004 - Train Accuracy: 87.28% - Test Accuracy: 89.12%


...

[Epoch 13] Train Loss: 0.000407 - Test Loss: 0.000755 - Train Accuracy: 98.75% - Test Accuracy: 97.67%



Test 13: 100%|██████████| 100/100 [00:02<00:00, 45.04batch/s]

[Epoch 14] Train Loss: 0.000358 - Test Loss: 0.000732 - Train Accuracy: 98.92% - Test Accuracy: 97.84%



Test 14: 100%|██████████| 100/100 [00:02<00:00, 45.63batch/s]

[Epoch 15] Train Loss: 0.000317 - Test Loss: 0.000759 - Train Accuracy: 99.05% - Test Accuracy: 97.74%

BEST TEST ACCURACY:  97.85  in epoch  11
{0.1: {'AdamW': {'best_accuracy': 89.12, 'best_epoch': 1}, 'SGD': {'best_accuracy': 98.41, 'best_epoch': 10}, 'Adadelta': {'best_accuracy': 97.44, 'best_epoch': 14}, 'Adam': {'best_accuracy': 91.37, 'best_epoch': 1}}, 0.01: {'AdamW': {'best_accuracy': 97.39, 'best_epoch': 14}, 'SGD': {'best_accuracy': 97.75, 'best_epoch': 14}, 'Adadelta': {'best_accuracy': 91.88, 'best_epoch': 14}, 'Adam': {'best_accuracy': 97.32, 'best_epoch': 14}}, 0.001: {'AdamW': {'best_accuracy': 98.51, 'best_epoch': 11}, 'SGD': {'best_accuracy': 92.51, 'best_epoch': 14}, 'Adadelta': {'best_accuracy': 73.24, 'best_epoch': 14}, 'Adam': {'best_accuracy': 98.53, 'best_epoch': 7}}, 0.0001: {'AdamW': {'best_accuracy': 97.83, 'best_epoch': 12}, 'SGD': {'best_accuracy': 72.96, 'best_epoch': 14}, 'Adadelta': {'best_accuracy': 44.73, 'best_epoch': 14}, 'Adam': {'best_accuracy': 97

In [10]:
print(best_results)

{0.1: {'AdamW': {'best_accuracy': 89.12, 'best_epoch': 1}, 'SGD': {'best_accuracy': 98.41, 'best_epoch': 10}, 'Adadelta': {'best_accuracy': 97.44, 'best_epoch': 14}, 'Adam': {'best_accuracy': 91.37, 'best_epoch': 1}}, 0.01: {'AdamW': {'best_accuracy': 97.39, 'best_epoch': 14}, 'SGD': {'best_accuracy': 97.75, 'best_epoch': 14}, 'Adadelta': {'best_accuracy': 91.88, 'best_epoch': 14}, 'Adam': {'best_accuracy': 97.32, 'best_epoch': 14}}, 0.001: {'AdamW': {'best_accuracy': 98.51, 'best_epoch': 11}, 'SGD': {'best_accuracy': 92.51, 'best_epoch': 14}, 'Adadelta': {'best_accuracy': 73.24, 'best_epoch': 14}, 'Adam': {'best_accuracy': 98.53, 'best_epoch': 7}}, 0.0001: {'AdamW': {'best_accuracy': 97.83, 'best_epoch': 12}, 'SGD': {'best_accuracy': 72.96, 'best_epoch': 14}, 'Adadelta': {'best_accuracy': 44.73, 'best_epoch': 14}, 'Adam': {'best_accuracy': 97.85, 'best_epoch': 11}}}
